In [1]:
import cudf as pd
import seaborn as sns
import cupy as np
import matplotlib.pyplot as plt
from cuml.model_selection import train_test_split
from cuml.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')
from cuml.ensemble import RandomForestClassifier as cuRFC
from cuml.metrics.confusion_matrix import confusion_matrix
from sklearn.metrics import accuracy_score
!pip install ipyplot &> /dev/null
print ("ipyplot installed sucessfully!!")
import matplotlib.pyplot as plt
import ipyplot
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [71]:
sample=pd.read_csv('../input/netflix/sample_submission.csv')

In [2]:
train_df=pd.read_csv('../input/netflix/train.csv')

In [3]:
test_df=pd.read_csv('../input/netflix/train.csv')

In [4]:
train_df.info()

In [5]:
categorical_var=[column for column in train_df.columns if (train_df[column].dtype=='object') | (len(train_df[column].unique())<=20) ]

In [6]:
train_df[categorical_var].head()

In [7]:
numerical_var=train_df.columns.drop(categorical_var)

In [8]:
train_df[numerical_var].head()

In [9]:
len(categorical_var)

In [10]:

train_df.head()

# target feature

In [11]:
train_df['target'].unique()

In [12]:
np.asarray(train_df['target'])

# percentage null values

In [13]:
null_values_columns=[]
percent_null_values=[]
for column in train_df.columns:
    percent_null_value=((train_df[column].isna().sum())/(len(train_df))) * 100
    if(percent_null_value > 0):
        null_values_columns.append(column)
        percent_null_values.append(percent_null_value)
df_null=pd.DataFrame(null_values_columns,columns=['column'])
df_null['percent_null_values']=percent_null_values
df_null.sort_values('percent_null_values')





# columns having zero variance (or only one class)

In [14]:
def zero_var(df1):
    zero_var_columns=[]
    for column in df1.columns:
        if(len(df1[column].unique())==1):
            zero_var_columns.append(column)
    return zero_var_columns

In [15]:
zero_var_columns=zero_var(train_df)

In [16]:
zero_var_columns

In [17]:
# many columns have zero variance, we will delete these while data cleaning

# Data preprocessing

In [18]:
# making a new copy so that the original data set remains unchanged
df=train_df.copy()

In [19]:
df.shape

In [20]:
# removing id column
df.drop('id',axis=1,inplace=True)

In [21]:

    df.drop(['feature_191','feature_192','feature_199','feature_201'],axis=1,inplace=True)


# Data cleaning

## 1. Dropping columns with very large no, of null values (greater than 10%)

In [22]:
df_null

In [23]:
large_null_columns=df_null[df_null['percent_null_values']>10]

In [24]:
large_null_columns

In [25]:
 df.drop(['feature_5','feature_6','feature_7','feature_8','feature_9','feature_10','feature_11','feature_12','feature_13','feature_75','feature_76','feature_83','feature_84','feature_85','feature_86','feature_153','feature_172','feature_193','feature_194','feature_195','feature_196','feature_197','feature_198','feature_200','feature_202','feature_203','feature_204','feature_255'],axis=1,inplace=True)

In [26]:
test_df.drop(['feature_5','feature_6','feature_7','feature_8','feature_9','feature_10','feature_11','feature_12','feature_13','feature_75','feature_76','feature_83','feature_84','feature_85','feature_86','feature_153','feature_172','feature_193','feature_194','feature_195','feature_196','feature_197','feature_198','feature_200','feature_202','feature_203','feature_204','feature_255'],axis=1,inplace=True)

In [27]:
df.shape

In [28]:
null_values_columns=[]
percent_null_values=[]
for column in df.columns:
    percent_null_value=((df[column].isna().sum())/(len(df))) * 100
    if(percent_null_value > 0):
        null_values_columns.append(column)
        percent_null_values.append(percent_null_value)
df_null=pd.DataFrame(null_values_columns,columns=['column'])
df_null['percent_null_values']=percent_null_values
df_null.sort_values('percent_null_values')



In [29]:
# successfully dropped the columns with large no. of na values

In [30]:
categorical_var=[column for column in df.columns if (df[column].dtype=='object')]
numerical_var=df.columns.drop(categorical_var)

In [31]:
df[numerical_var]

In [32]:
df[categorical_var].head()

In [34]:
for column in categorical_var:
    print(column,' = ',len(df[column].unique()),'\t')

# dropping categorical columns having too many classes

In [36]:
def remove_more_than_80(df2):
    removed_columns_80=[]
    for column in categorical_var:
        if(len(df[column].unique())>70):
            df2.drop(column, axis=1, inplace=True)
#             categorical_var.remove(column)
            removed_columns_80.append(column)
    print('removed columns are:\n',removed_columns_80)
    return removed_columns_80

In [37]:
removed_columns_80=remove_more_than_80(df)

In [38]:
test_df.drop(removed_columns_80,axis=1,inplace=True)

In [39]:
categorical_var

In [40]:
#removing the the deleted elements from categorical_var list
for value in removed_columns_80:
    categorical_var.remove(value)

In [41]:
for column in categorical_var:
    print(column,' = ',len(df[column].unique()),end='                                                    ')

In [43]:
df.shape

In [44]:
df.head()

# label encoding 


In [45]:
def label_encoding(df1,df2):
    le=LabelEncoder()
    new=le.fit_transform(df1)
    
    df2.loc[~df2.isin(le.classes_)] = -1    
    df2.loc[df2.isin(le.classes_)] = le.transform(df2[df2.isin(le.classes_)])

    
#     new_test=le.transform(df2)
    return new,df2

In [46]:
# le_models=[]
for column in categorical_var:
    new,new_test=label_encoding(df[column],test_df[column])
    df[column]=new
    test_df[column]=new_test

In [47]:
test_df.head()

In [48]:
df[categorical_var]

In [49]:
test_df

# filling na values with -1 

In [50]:
df.head()

In [51]:
df.fillna(-1,inplace=True)

In [52]:
test_df.fillna(-1,inplace=True)

In [53]:

X=df.drop('target',axis=1)
Y=df['target']
Y=np.array(Y)
Y=Y.reshape(-1,1)

In [54]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X.as_matrix())

# Train test split

In [55]:
#X_train, X_test, y_train, y_test =train_test_split(df.drop('target',axis=1),df['target'],test_size=0.2,stratify=df['target'])

In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.3, random_state=42)

In [57]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [58]:
y_train = y_train.reshape(len(y_train),1)
y_test = y_test.reshape(len(y_test),1)
print(y_train.shape)
print(y_test.shape)

In [59]:
import cuml
RFC_model = cuRFC(max_features=1.0,
                   n_bins=10,
                   n_estimators=40)
RFC_model.fit(X_train.astype('float64'),y_train.astype('float64'))
RFC_predict = RFC_model.predict(X_test.astype('float64'))

In [60]:
y_test=y_test.get()

In [61]:
y_test = y_test.reshape(1,len(y_test))[0]
print(y_test)
print(type(y_test))

In [62]:
print(RFC_predict)
print(type(RFC_predict))

In [63]:
accuracy_score(y_test,RFC_predict)

In [64]:
confusion_matrix(y_test,RFC_predict)

In [65]:
print(classification_report(y_test,RFC_predict,zero_division=0))